## Code used to train the RapidEye regular models 

The models were trained in Google Colaboratory Virtual Environment, thus, to work properly, this notebook should be loaded in google drive.

* [32x32 models](#32-x-32-models) 

* [64x64 models](#64-x-64-models) 

* [128x128 models](#128-x-128-models) 

In [ ]:
# Import libraries

import tensorflow as  tf
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
from packaging import version

%tensorflow_version 2.x
from tensorflow import keras
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

In [ ]:
# Install segmetation_models library (https://github.com/qubvel/segmentation_models)
pip install segmentation_models

In [ ]:
# Load segmentation)models library 
import segmentation_models as sm

#### 32 x 32 models

In [ ]:
# Load training data 32x32 - regular - the Strings are the directions to the .npy files in google drive
X_train = np.load("/content/drive/My Drive/Mestrado/artigo/artigo_final/data/non_augmented/32_32/regular/arrays/X_train_32_regular.npy")
Y_train = np.load("/content/drive/My Drive/Mestrado/artigo/artigo_final/data/non_augmented/32_32/regular/arrays/Y_train_32_regular.npy")

# Load test data - Area 1
X_test_area_1 = np.load("/content/drive/My Drive/Mestrado/artigo/artigo_final/data/test/test_without_terrain/area_1/arrays/X_test_test_area_1.npy")
Y_test_area_1 = np.load("/content/drive/My Drive/Mestrado/artigo/artigo_final/data/test/test_without_terrain/area_1/arrays/Y_test_test_area_1.npy")

# Load test data - Area 2
X_test_area_2 = np.load("/content/drive/My Drive/Mestrado/artigo/artigo_final/data/test/test_without_terrain/area_2/arrays/X_test_test_area_2.npy")
Y_test_area_2 = np.load("/content/drive/My Drive/Mestrado/artigo/artigo_final/data/test/test_without_terrain/area_2/arrays/Y_test_test_area_2.npy")

In [ ]:
# Evaluate data dimensions
print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}\nX_test_area_1 shape: {X_test_area_1.shape}, Y_test_area_1 shape: {Y_test_area_1.shape},\nX_test_area_2 shape: {X_test_area_2.shape}, Y_test_area_2 shape: {Y_test_area_2.shape}")

In [ ]:
# Evaluation Metrics - Precision, Recall, FScore, IoU
metrics = [sm.metrics.Precision(threshold=0.5),sm.metrics.Recall(threshold=0.5),sm.metrics.FScore(threshold=0.5,beta=1),sm.metrics.IOUScore(threshold=0.5)]

In [ ]:
# Unet Architecture
def Unet_Original(lr,filtersFirstLayer, pretrained_weights = None,input_size = (32,32,5)):
    inputs = Input(input_size)
    conv1 = Conv2D(filtersFirstLayer, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(inputs)
    conv1 = Conv2D(filtersFirstLayer, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(filtersFirstLayer*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(pool1)
    conv2 = Conv2D(filtersFirstLayer*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(filtersFirstLayer*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(pool2)
    conv3 = Conv2D(filtersFirstLayer*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(filtersFirstLayer*8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(pool3)
    conv4 = Conv2D(filtersFirstLayer*8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(filtersFirstLayer*16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(pool4)
    conv5 = Conv2D(filtersFirstLayer*16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(conv5)

    up6 = Conv2D(filtersFirstLayer*8, 2, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(UpSampling2D(size = (2,2))(conv5))
    merge6 = concatenate([conv4,up6], axis = 3)
    conv6 = Conv2D(filtersFirstLayer*8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(merge6)
    conv6 = Conv2D(filtersFirstLayer*8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(conv6)

    up7 = Conv2D(filtersFirstLayer*4, 2, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(filtersFirstLayer*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(merge7)
    conv7 = Conv2D(filtersFirstLayer*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(conv7)

    up8 = Conv2D(filtersFirstLayer*2, 2, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(filtersFirstLayer*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(merge8)
    conv8 = Conv2D(filtersFirstLayer*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(conv8)

    up9 = Conv2D(filtersFirstLayer, 2, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(filtersFirstLayer, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(merge9)
    conv9 = Conv2D(filtersFirstLayer, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs, conv10)

    model.compile(optimizer = Adam(lr = lr), loss = 'binary_crossentropy', metrics = metrics)
    
    model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

In [ ]:
# Model training - Results are saved in a .csv file

# size of the tiles
size = 32
# Sampling method
sampling = "regular"
# number of filters 
filters = [16,32,64]
# lr = 0.001
lr = [10e-4]
# batch sizes 
batch_size = [16,32,64,128]

# dictionary that will save the results
dic = {}

# Hyperparameters
dic["model"] = []
dic["batch_size"] = []
dic["learning_rate"] = []
dic["filters"] = []

# test area 1
dic["precision_area_1"] = []
dic["recall_area_1"] = []
dic["f1_score_area_1"] = []
dic["iou_score_area_1"] = []

# test area 2
dic["precision_area_2"] = []
dic["recall_area_2"] = []
dic["f1_score_area_2"] = []
dic["iou_score_area_2"] = []




# loop over all the filters in the filter list
for fiilter in filters:
    # loop over the learning rates (used to evalute 0.01 and 0.0001 without good results)
    for learning_rate in lr:
        # loop over all batch sizes in batch_size list
        for batch in batch_size:
            # load the model
            model = Unet_Original(filtersFirstLayer= fiilter, lr = learning_rate)
            # Save the models only when validation loss decrease
            model_checkpoint = tf.keras.callbacks.ModelCheckpoint(f'/content/drive/My Drive/Mestrado/artigo/artigo_final/results/non_augmented/{size}_{size}/{sampling}/model/unet/unet_{sampling}_size_{size}_filters_{fiilter}_batch_size_{batch}_lr_{learning_rate}.hdf5', monitor='val_loss', mode='min',verbose=1, save_best_only=True,save_weights_only = True)
            # Stop after 20 epochs without decreasing the validation loss
            early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
            print(fiilter, learning_rate,batch)
            # fit the model 30% of the dataset was used as validation
            history = model.fit(X_train,Y_train,batch_size = batch,epochs=200,validation_split=0.3,callbacks=[model_checkpoint, early_stopping])

            # summarize history for iou score
            plt.plot(history.history['iou_score'])
            plt.plot(history.history['val_iou_score'])
            plt.title('model accuracy')
            plt.ylabel('accuracy')
            plt.xlabel('epoch')
            plt.legend(['train', 'validation'], loc='upper left')
            # save plots 
            plt.savefig(f"/content/drive/My Drive/Mestrado/artigo/artigo_final/results/non_augmented/{size}_{size}/{sampling}/plots/unet/unet_{sampling}_size_{size}_filters_{fiilter}_batch_size_{batch}_lr_{learning_rate}_iou_score.png")
            plt.show()
            # summarize history for loss
            plt.plot(history.history['loss'])
            plt.plot(history.history['val_loss'])
            plt.title('model loss')
            plt.ylabel('loss')
            plt.xlabel('epoch')
            plt.legend(['train', 'validation'], loc='upper left')
            plt.savefig(f"/content/drive/My Drive/Mestrado/artigo/artigo_final/results/non_augmented/{size}_{size}/{sampling}/plots/unet/unet_{sampling}_size_{size}_filters_{fiilter}_batch_size_{batch}_lr_{learning_rate}_val_loss.png")
            plt.show()
            
            # load unet to evaluate the test data
            unet_original = Unet_Original(filtersFirstLayer= fiilter, lr = learning_rate,input_size=(1024,1024,5))
            # load the last saved weight from the training
            unet_original.load_weights(f"/content/drive/My Drive/Mestrado/artigo/artigo_final/results/non_augmented/{size}_{size}/{sampling}/model/unet/unet_{sampling}_size_{size}_filters_{fiilter}_batch_size_{batch}_lr_{learning_rate}.hdf5")
            
           # Evaluate test area 1
            res_1 = unet_original.evaluate(X_test_area_1,Y_test_area_1)
            
            # Evaluate test area 2
            res_2 = unet_original.evaluate(X_test_area_2,Y_test_area_2)

            # Data to plot the predicted output
            preds_train_1 = unet_original.predict(X_test_area_1, verbose=1)
            preds_train_t1 = (preds_train_1 > 0.5).astype(np.uint8)
            preds_train_2 = unet_original.predict(X_test_area_2, verbose=1)
            preds_train_t2 = (preds_train_2 > 0.5).astype(np.uint8)


            # save results on the dictionary
            dic["model"].append("Unet")
            dic["batch_size"].append(batch)
            dic["learning_rate"].append(learning_rate)
            dic["filters"].append(fiilter)
            dic["precision_area_1"].append(res_1[1])
            dic["recall_area_1"].append(res_1[2])
            dic["f1_score_area_1"].append(res_1[3])
            dic["iou_score_area_1"].append(res_1[4])
           
            dic["precision_area_2"].append(res_2[1])
            dic["recall_area_2"].append(res_2[2])
            dic["f1_score_area_2"].append(res_2[3])
            dic["iou_score_area_2"].append(res_2[4])
            
    
            # Plot the results and save the plots
            f, axarr = plt.subplots(2,3,figsize=(10,10))
            axarr[0,0].imshow(X_test_area_1[0][:,:,:3])
            axarr[0,1].imshow(np.squeeze(preds_train_t1[0]))
            axarr[0,2].imshow(np.squeeze(Y_test_area_1[0]))
            axarr[1,0].imshow(X_test_area_2[0][:,:,:3])
            axarr[1,1].imshow(np.squeeze(preds_train_t2[0]))
            axarr[1,2].imshow(np.squeeze(Y_test_area_2[0]))
            f.savefig(f"/content/drive/My Drive/Mestrado/artigo/artigo_final/results/non_augmented/{size}_{size}/{sampling}/images/unet/unet_{sampling}_size_{size}_filters_{fiilter}_batch_size_{batch}_lr_{learning_rate}_result.png")
      
            # Convert results to a dataframe
            results = pd.DataFrame(dic)
            # Export as csv
            results.to_csv(f'/content/drive/My Drive/Mestrado/artigo/artigo_final/results/non_augmented/{size}_{size}/{sampling}/result_table/unet/unet_{sampling}_size_{size}_filters_{fiilter}_batch_size_{batch}.csv', index = False)

#### 64 x 64 models

In [ ]:
# Load training data 32x32 - regular - the Strings are the directions to the .npy files in google drive
X_train = np.load("/content/drive/My Drive/Mestrado/artigo/artigo_final/data/non_augmented/64_64/regular/arrays/X_train_64_regular.npy")
Y_train = np.load("/content/drive/My Drive/Mestrado/artigo/artigo_final/data/non_augmented/64_64/regular/arrays/Y_train_64_regular.npy")

In [ ]:
# Evaluate data dimensions
print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}\nX_test_area_1 shape: {X_test_area_1.shape}, Y_test_area_1 shape: {Y_test_area_1.shape},\nX_test_area_2 shape: {X_test_area_2.shape}, Y_test_area_2 shape: {Y_test_area_2.shape}")

In [ ]:
# Model training - Results are saved in a .csv file

# size of the tiles
size = 64
# Sampling method
sampling = "regular"
# number of filters 
filters = [16,32,64]
# lr = 0.001
lr = [10e-4]
# batch sizes 
batch_size = [16,32,64,128]

# dictionary that will save the results
dic = {}

# Hyperparameters
dic["model"] = []
dic["batch_size"] = []
dic["learning_rate"] = []
dic["filters"] = []

# test area 1
dic["precision_area_1"] = []
dic["recall_area_1"] = []
dic["f1_score_area_1"] = []
dic["iou_score_area_1"] = []

# test area 2
dic["precision_area_2"] = []
dic["recall_area_2"] = []
dic["f1_score_area_2"] = []
dic["iou_score_area_2"] = []




# loop over all the filters in the filter list
for fiilter in filters:
    # loop over the learning rates (used to evalute 0.01 and 0.0001 without good results)
    for learning_rate in lr:
        # loop over all batch sizes in batch_size list
        for batch in batch_size:
            # load the model
            model = Unet_Original(filtersFirstLayer= fiilter, lr = learning_rate, input_size = (64,64,5))
            # Save the models only when validation loss decrease
            model_checkpoint = tf.keras.callbacks.ModelCheckpoint(f'/content/drive/My Drive/Mestrado/artigo/artigo_final/results/non_augmented/{size}_{size}/{sampling}/model/unet/unet_{sampling}_size_{size}_filters_{fiilter}_batch_size_{batch}_lr_{learning_rate}.hdf5', monitor='val_loss', mode='min',verbose=1, save_best_only=True,save_weights_only = True)
            # Stop after 20 epochs without decreasing the validation loss
            early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
            print(fiilter, learning_rate,batch)
            # fit the model 30% of the dataset was used as validation
            history = model.fit(X_train,Y_train,batch_size = batch,epochs=200,validation_split=0.3,callbacks=[model_checkpoint, early_stopping])

            # summarize history for iou score
            plt.plot(history.history['iou_score'])
            plt.plot(history.history['val_iou_score'])
            plt.title('model accuracy')
            plt.ylabel('accuracy')
            plt.xlabel('epoch')
            plt.legend(['train', 'validation'], loc='upper left')
            # save plots 
            plt.savefig(f"/content/drive/My Drive/Mestrado/artigo/artigo_final/results/non_augmented/{size}_{size}/{sampling}/plots/unet/unet_{sampling}_size_{size}_filters_{fiilter}_batch_size_{batch}_lr_{learning_rate}_iou_score.png")
            plt.show()
            # summarize history for loss
            plt.plot(history.history['loss'])
            plt.plot(history.history['val_loss'])
            plt.title('model loss')
            plt.ylabel('loss')
            plt.xlabel('epoch')
            plt.legend(['train', 'validation'], loc='upper left')
            plt.savefig(f"/content/drive/My Drive/Mestrado/artigo/artigo_final/results/non_augmented/{size}_{size}/{sampling}/plots/unet/unet_{sampling}_size_{size}_filters_{fiilter}_batch_size_{batch}_lr_{learning_rate}_val_loss.png")
            plt.show()
            
            # load unet to evaluate the test data
            unet_original = Unet_Original(filtersFirstLayer= fiilter, lr = learning_rate,input_size=(1024,1024,5))
            # load the last saved weight from the training
            unet_original.load_weights(f"/content/drive/My Drive/Mestrado/artigo/artigo_final/results/non_augmented/{size}_{size}/{sampling}/model/unet/unet_{sampling}_size_{size}_filters_{fiilter}_batch_size_{batch}_lr_{learning_rate}.hdf5")
            
           # Evaluate test area 1
            res_1 = unet_original.evaluate(X_test_area_1,Y_test_area_1)
            
            # Evaluate test area 2
            res_2 = unet_original.evaluate(X_test_area_2,Y_test_area_2)

            # Data to plot the predicted output
            preds_train_1 = unet_original.predict(X_test_area_1, verbose=1)
            preds_train_t1 = (preds_train_1 > 0.5).astype(np.uint8)
            preds_train_2 = unet_original.predict(X_test_area_2, verbose=1)
            preds_train_t2 = (preds_train_2 > 0.5).astype(np.uint8)


            # save results on the dictionary
            dic["model"].append("Unet")
            dic["batch_size"].append(batch)
            dic["learning_rate"].append(learning_rate)
            dic["filters"].append(fiilter)
            dic["precision_area_1"].append(res_1[1])
            dic["recall_area_1"].append(res_1[2])
            dic["f1_score_area_1"].append(res_1[3])
            dic["iou_score_area_1"].append(res_1[4])
           
            dic["precision_area_2"].append(res_2[1])
            dic["recall_area_2"].append(res_2[2])
            dic["f1_score_area_2"].append(res_2[3])
            dic["iou_score_area_2"].append(res_2[4])
            
    
            # Plot the results and save the plots
            f, axarr = plt.subplots(2,3,figsize=(10,10))
            axarr[0,0].imshow(X_test_area_1[0][:,:,:3])
            axarr[0,1].imshow(np.squeeze(preds_train_t1[0]))
            axarr[0,2].imshow(np.squeeze(Y_test_area_1[0]))
            axarr[1,0].imshow(X_test_area_2[0][:,:,:3])
            axarr[1,1].imshow(np.squeeze(preds_train_t2[0]))
            axarr[1,2].imshow(np.squeeze(Y_test_area_2[0]))
            f.savefig(f"/content/drive/My Drive/Mestrado/artigo/artigo_final/results/non_augmented/{size}_{size}/{sampling}/images/unet/unet_{sampling}_size_{size}_filters_{fiilter}_batch_size_{batch}_lr_{learning_rate}_result.png")
      
            # Convert results to a dataframe
            results = pd.DataFrame(dic)
            # Export as csv
            results.to_csv(f'/content/drive/My Drive/Mestrado/artigo/artigo_final/results/non_augmented/{size}_{size}/{sampling}/result_table/unet/unet_{sampling}_size_{size}_filters_{fiilter}_batch_size_{batch}.csv', index = False)

#### 128 x 128 models

In [ ]:
# Load training data 32x32 - regular - the Strings are the directions to the .npy files in google drive
X_train = np.load("/content/drive/My Drive/Mestrado/artigo/artigo_final/data/non_augmented/128_128/regular/arrays/X_train_128_regular.npy")
Y_train = np.load("/content/drive/My Drive/Mestrado/artigo/artigo_final/data/non_augmented/128_128/regular/arrays/Y_train_128_regular.npy")

In [ ]:
# Evaluate data dimensions
print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}\nX_test_area_1 shape: {X_test_area_1.shape}, Y_test_area_1 shape: {Y_test_area_1.shape},\nX_test_area_2 shape: {X_test_area_2.shape}, Y_test_area_2 shape: {Y_test_area_2.shape}")

In [ ]:
# Model training - Results are saved in a .csv file

# size of the tiles
size = 128
# Sampling method
sampling = "regular"
# number of filters 
filters = [16,32,64]
# lr = 0.001
lr = [10e-4]
# batch sizes 
batch_size = [16,32,64,128]

# dictionary that will save the results
dic = {}

# Hyperparameters
dic["model"] = []
dic["batch_size"] = []
dic["learning_rate"] = []
dic["filters"] = []

# test area 1
dic["precision_area_1"] = []
dic["recall_area_1"] = []
dic["f1_score_area_1"] = []
dic["iou_score_area_1"] = []

# test area 2
dic["precision_area_2"] = []
dic["recall_area_2"] = []
dic["f1_score_area_2"] = []
dic["iou_score_area_2"] = []




# loop over all the filters in the filter list
for fiilter in filters:
    # loop over the learning rates (used to evalute 0.01 and 0.0001 without good results)
    for learning_rate in lr:
        # loop over all batch sizes in batch_size list
        for batch in batch_size:
            # load the model
            model = Unet_Original(filtersFirstLayer= fiilter, lr = learning_rate, input_size = (128,128,5))
            # Save the models only when validation loss decrease
            model_checkpoint = tf.keras.callbacks.ModelCheckpoint(f'/content/drive/My Drive/Mestrado/artigo/artigo_final/results/non_augmented/{size}_{size}/{sampling}/model/unet/unet_{sampling}_size_{size}_filters_{fiilter}_batch_size_{batch}_lr_{learning_rate}.hdf5', monitor='val_loss', mode='min',verbose=1, save_best_only=True,save_weights_only = True)
            # Stop after 20 epochs without decreasing the validation loss
            early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
            print(fiilter, learning_rate,batch)
            # fit the model 30% of the dataset was used as validation
            history = model.fit(X_train,Y_train,batch_size = batch,epochs=200,validation_split=0.3,callbacks=[model_checkpoint, early_stopping])

            # summarize history for iou score
            plt.plot(history.history['iou_score'])
            plt.plot(history.history['val_iou_score'])
            plt.title('model accuracy')
            plt.ylabel('accuracy')
            plt.xlabel('epoch')
            plt.legend(['train', 'validation'], loc='upper left')
            # save plots 
            plt.savefig(f"/content/drive/My Drive/Mestrado/artigo/artigo_final/results/non_augmented/{size}_{size}/{sampling}/plots/unet/unet_{sampling}_size_{size}_filters_{fiilter}_batch_size_{batch}_lr_{learning_rate}_iou_score.png")
            plt.show()
            # summarize history for loss
            plt.plot(history.history['loss'])
            plt.plot(history.history['val_loss'])
            plt.title('model loss')
            plt.ylabel('loss')
            plt.xlabel('epoch')
            plt.legend(['train', 'validation'], loc='upper left')
            plt.savefig(f"/content/drive/My Drive/Mestrado/artigo/artigo_final/results/non_augmented/{size}_{size}/{sampling}/plots/unet/unet_{sampling}_size_{size}_filters_{fiilter}_batch_size_{batch}_lr_{learning_rate}_val_loss.png")
            plt.show()
            
            # load unet to evaluate the test data
            unet_original = Unet_Original(filtersFirstLayer= fiilter, lr = learning_rate,input_size=(1024,1024,5))
            # load the last saved weight from the training
            unet_original.load_weights(f"/content/drive/My Drive/Mestrado/artigo/artigo_final/results/non_augmented/{size}_{size}/{sampling}/model/unet/unet_{sampling}_size_{size}_filters_{fiilter}_batch_size_{batch}_lr_{learning_rate}.hdf5")
            
           # Evaluate test area 1
            res_1 = unet_original.evaluate(X_test_area_1,Y_test_area_1)
            
            # Evaluate test area 2
            res_2 = unet_original.evaluate(X_test_area_2,Y_test_area_2)

            # Data to plot the predicted output
            preds_train_1 = unet_original.predict(X_test_area_1, verbose=1)
            preds_train_t1 = (preds_train_1 > 0.5).astype(np.uint8)
            preds_train_2 = unet_original.predict(X_test_area_2, verbose=1)
            preds_train_t2 = (preds_train_2 > 0.5).astype(np.uint8)


            # save results on the dictionary
            dic["model"].append("Unet")
            dic["batch_size"].append(batch)
            dic["learning_rate"].append(learning_rate)
            dic["filters"].append(fiilter)
            dic["precision_area_1"].append(res_1[1])
            dic["recall_area_1"].append(res_1[2])
            dic["f1_score_area_1"].append(res_1[3])
            dic["iou_score_area_1"].append(res_1[4])
           
            dic["precision_area_2"].append(res_2[1])
            dic["recall_area_2"].append(res_2[2])
            dic["f1_score_area_2"].append(res_2[3])
            dic["iou_score_area_2"].append(res_2[4])
            
    
            # Plot the results and save the plots
            f, axarr = plt.subplots(2,3,figsize=(10,10))
            axarr[0,0].imshow(X_test_area_1[0][:,:,:3])
            axarr[0,1].imshow(np.squeeze(preds_train_t1[0]))
            axarr[0,2].imshow(np.squeeze(Y_test_area_1[0]))
            axarr[1,0].imshow(X_test_area_2[0][:,:,:3])
            axarr[1,1].imshow(np.squeeze(preds_train_t2[0]))
            axarr[1,2].imshow(np.squeeze(Y_test_area_2[0]))
            f.savefig(f"/content/drive/My Drive/Mestrado/artigo/artigo_final/results/non_augmented/{size}_{size}/{sampling}/images/unet/unet_{sampling}_size_{size}_filters_{fiilter}_batch_size_{batch}_lr_{learning_rate}_result.png")
      
            # Convert results to a dataframe
            results = pd.DataFrame(dic)
            # Export as csv
            results.to_csv(f'/content/drive/My Drive/Mestrado/artigo/artigo_final/results/non_augmented/{size}_{size}/{sampling}/result_table/unet/unet_{sampling}_size_{size}_filters_{fiilter}_batch_size_{batch}.csv', index = False)